<div style="text-align: right;">
-kngbq
</div>


In [21]:
import os
import json
import subprocess
import time
from mininet.net import Mininet
from mininet.topo import Topo
from mininet.node import Switch
from mininet.cli import CLI
from mininet.log import setLogLevel, info

# Custom BMv2 P4 switch
class P4Switch(Switch):
    def __init__(self, name, json_file, thrift_port=9090, **params):
        Switch.__init__(self, name, **params)
        self.json_file = json_file
        self.thrift_port = thrift_port

    def start(self, controllers):
        cmd = f"simple_switch --thrift-port {self.thrift_port} --log-console {self.json_file} &"
        info(f"*** Starting P4 switch {self.name}: {cmd}\n")
        os.system(cmd)
        time.sleep(2)  # Wait for the switch to start

    def stop(self):
        info(f"*** Stopping P4 switch {self.name}\n")
        os.system("pkill simple_switch")

# Mininet topology (h1 ↔ s1 ↔ h2)
class NetworkTopo(Topo):
    def __init__(self, json_file):
        Topo.__init__(self)

        s1 = self.addSwitch('s1', cls=P4Switch, json_file=json_file, thrift_port=9090)
        h1 = self.addHost('h1', ip='10.0.1.1/24', mac='00:00:00:00:01:01')
        h2 = self.addHost('h2', ip='10.0.1.2/24', mac='00:00:00:00:01:02')

        self.addLink(h1, s1)
        self.addLink(h2, s1)

def compile_p4():
    print("Compiling P4 program...")
    result = subprocess.run(
        ['p4c', '--target', 'bmv2', '--arch', 'v1model', '-o', '.', 'forward.p4'], 
        capture_output=True, text=True
    )

    if result.returncode != 0:
        print("P4 compilation failed!")
        print(f"Error: {result.stderr}")
        exit(1)
    else:
        print("P4 compilation successful")

def validate_json():
    try:
        with open('forward.json', 'r') as f:
            json.load(f)
        print("JSON output is valid")
    except (json.JSONDecodeError, FileNotFoundError) as e:
        print(f"Error with JSON output: {e}")
        exit(1)

def start_network():
    json_file = "forward.json"  
    net = Mininet(topo=NetworkTopo(json_file), controller=None)
    
    net.start()
    
    # Configure multicast flooding and table entries
    s1 = net.get('s1')
    s1.cmd('simple_switch_CLI --thrift-port 9090 << EOF\n'
           # Multicast group configuration
           'mc_mgrp_create 1\n'
           'mc_node_create 0 0 1\n'  # Ports 0 and 1
           'mc_node_associate 1 0\n'
           
           # Ethernet table entries
           'table_add ethernet_exact forward 00:00:00:00:01:01 => 0\n'
           'table_add ethernet_exact forward 00:00:00:00:01:02 => 1\n'
           
           # IPv4 table entries
           'table_add ipv4_lpm forward 10.0.1.1/32 => 0\n'
           'table_add ipv4_lpm forward 10.0.1.2/32 => 1\n'
           'EOF')
    
    # Configure hosts
    h1, h2 = net.get('h1', 'h2')
    
    # Populate ARP tables
    h1.cmd('arp -s 10.0.1.2 00:00:00:00:01:02')
    h2.cmd('arp -s 10.0.1.1 00:00:00:00:01:01')
    
    print("*** Running ping test ***")
    net.pingAll()
    
    CLI(net)
    net.stop()

if __name__ == '__main__':
    setLogLevel('info')

    # Clean up before running
    os.system('sudo pkill -f simple_switch')
    os.system('sudo rm -f *.log *.txt')

    # Verify installations
    for tool in ['p4c', 'simple_switch']:
        if subprocess.run(['which', tool], capture_output=True).returncode != 0:
            print(f"ERROR: {tool} not found in PATH!")
            exit(1)

    compile_p4()      # Compile P4 file
    validate_json()   # Ensure JSON is valid
    start_network()   # Start Mininet

Compiling P4 program...


*** Creating network
*** Adding hosts:
h1 h2 
*** Adding switches:
s1 
*** Adding links:
(h1, s1) (h2, s1) 
*** Configuring hosts
h1 h2 
*** Starting controller

*** Starting 1 switches
s1 *** Starting P4 switch s1: simple_switch --thrift-port 9090 --log-console forward.json &


P4 compilation successful
JSON output is valid
Calling target program-options parser
[17:18:12.973] [bmv2] [D] [thread 38550] Set default default entry for table 'MyIngress.ethernet_exact': MyIngress.drop - 
[17:18:12.973] [bmv2] [D] [thread 38550] Set default default entry for table 'MyIngress.ipv4_lpm': MyIngress.drop - 
[17:18:12.973] [bmv2] [I] [thread 38550] Starting Thrift server on port 9090
[17:18:12.973] [bmv2] [I] [thread 38550] Thrift server was started



*** Ping: testing ping reachability
h1 -> 

[17:18:15.023] [bmv2] [T] [thread 38564] bm_get_config
[17:18:15.024] [bmv2] [T] [thread 38564] bm_mc_mgrp_create
[17:18:15.024] [bmv2] [D] [thread 38564] mgrp node created for mgid 1
[17:18:15.025] [bmv2] [T] [thread 38564] bm_mc_node_create
[17:18:15.025] [bmv2] [D] [thread 38564] node created for rid 0
[17:18:15.025] [bmv2] [T] [thread 38564] bm_mc_node_associate
[17:18:15.025] [bmv2] [D] [thread 38564] node associated with mgid 1
[17:18:15.025] [bmv2] [T] [thread 38564] bm_table_add_entry
[17:18:15.025] [bmv2] [D] [thread 38564] Entry 0 added to table 'MyIngress.ethernet_exact'
[17:18:15.025] [bmv2] [D] [thread 38564] Dumping entry 0
Match key:
* hdr.ethernet.dstAddr: EXACT     000000000101
Action entry: MyIngress.forward - 0,

[17:18:15.025] [bmv2] [T] [thread 38564] bm_table_add_entry
[17:18:15.025] [bmv2] [D] [thread 38564] Entry 1 added to table 'MyIngress.ethernet_exact'
[17:18:15.025] [bmv2] [D] [thread 38564] Dumping entry 1
Match key:
* hdr.ethernet.dstAddr: EXACT     00000

X 
h2 -> X 
*** Results: 100% dropped (0/2 received)
*** Starting CLI:


mininet>  exit


*** Stopping 0 controllers

*** Stopping 2 links
..
*** Stopping 1 switches
s1 *** Stopping P4 switch s1

*** Stopping 2 hosts
h1 h2 
*** Done


In [ ]:
# !pip install p4runtime

#!pip list
#!pip install ryu

In [1]:
import sys
import os

# Remove duplicate paths
sys.path = list(set(sys.path))

# Check if the executables are available
os.system('simple_switch --version')
os.system('p4c --version')
os.system('ryu --version')
os.system("ryu-manager ryu.app.simple_switch_13")


1.15.0-892c4219
p4c 1.2.5.3 (SHA: 1af5c6a60 BUILD: Release)
ryu 4.34


Traceback (most recent call last):
  File "/usr/local/bin/ryu-manager", line 5, in <module>
    from ryu.cmd.manager import main
  File "/usr/local/lib/python3.10/dist-packages/ryu/cmd/manager.py", line 22, in <module>
    from ryu.lib import hub
  File "/usr/local/lib/python3.10/dist-packages/ryu/lib/hub.py", line 30, in <module>
    import eventlet
  File "/usr/local/lib/python3.10/dist-packages/eventlet/__init__.py", line 17, in <module>
    from eventlet import convenience
  File "/usr/local/lib/python3.10/dist-packages/eventlet/convenience.py", line 7, in <module>
    from eventlet.green import socket
  File "/usr/local/lib/python3.10/dist-packages/eventlet/green/socket.py", line 21, in <module>
    from eventlet.support import greendns
  File "/usr/local/lib/python3.10/dist-packages/eventlet/support/greendns.py", line 66, in <module>
    setattr(dns, pkg, import_patched('dns.' + pkg))
  File "/usr/local/lib/python3.10/dist-packages/eventlet/support/greendns.py", line 61, in impor

256

In [2]:
import sys
print(sys.executable)


/home/ubuntu/miniconda3/envs/p4-env/bin/python


In [ ]:
from mininet.net import Mininet
from mininet.topo import Topo
from mininet.node import Host
from mininet.link import TCLink

class SimpleTopo(Topo):
    def __init__(self):
        Topo.__init__(self)
        
        # Add 3 switches
        s1 = self.addSwitch('s1', cls=None, cmd='simple_switch --interface 0@veth0 --log-console basic_forward.json')
        s2 = self.addSwitch('s2', cls=None, cmd='simple_switch --interface 0@veth2 --log-console basic_forward.json')
        s3 = self.addSwitch('s3', cls=None, cmd='simple_switch --interface 0@veth4 --log-console basic_forward.json')
        
        # Add 3 hosts
        h1 = self.addHost('h1', ip='10.0.1.1/24')
        h2 = self.addHost('h2', ip='10.0.1.2/24')
        h3 = self.addHost('h3', ip='10.0.1.3/24')
        
        # Connect hosts to switches
        self.addLink(h1, s1)
        self.addLink(h2, s2)
        self.addLink(h3, s3)
        
        # Connect switches in a line: s1 <-> s2 <-> s3
        self.addLink(s1, s2)
        self.addLink(s2, s3)

# Create network
net = Mininet(topo=SimpleTopo(), controller=None)
net.start()

# Open Mininet CLI for manual control (optional)
net.interact()

In [ ]:
##### P4 program behavior in all 3 switches

In [ ]:
from mininet.net import Mininet
from mininet.topo import Topo
import os

class SimpleTopo(Topo):
    def __init__(self):
        Topo.__init__(self)
        
        # Get absolute path to JSON file
        json_path = os.path.abspath("basic_forward.json")
        
        # Add switches with explicit Thrift ports
        s1 = self.addSwitch('s1', cls=None, cmd='/usr/local/bin/simple_switch --thrift-port 9090 --log-console %s' % json_path)
        s2 = self.addSwitch('s2', cls=None, cmd=f'simple_switch --thrift-port 9091 --log-console {json_path}')
        s3 = self.addSwitch('s3', cls=None, cmd=f'simple_switch --thrift-port 9092 --log-console {json_path}')
        
        # Add hosts
        h1 = self.addHost('h1', ip='10.0.1.1/24')
        h2 = self.addHost('h2', ip='10.0.1.2/24')
        h3 = self.addHost('h3', ip='10.0.1.3/24')
        
        # Connect hosts to switches
        self.addLink(h1, s1)
        self.addLink(h2, s2)
        self.addLink(h3, s3)
        
        # Connect switches in a line
        self.addLink(s1, s2)
        self.addLink(s2, s3)

# Start the network
net = Mininet(topo=SimpleTopo(), controller=None)
net.start()
net.interact()  # Keep the network running

In [ ]:
from mininet.net import Mininet
from mininet.topo import Topo
import os
from mininet.node import Controller, OVSController, OVSSwitch
from mininet.cli import CLI
from mininet.link import TCLink

class SimpleTopo(Topo):
    def __init__(self):
        Topo.__init__(self)

        # Get absolute path to JSON file
        json_path = os.path.abspath("basic_forward.json")

        # Add switches
        s1 = self.addSwitch('s1', cls=OVSSwitch)
        s2 = self.addSwitch('s2', cls=OVSSwitch)  # Default OVS switch
        s3 = self.addSwitch('s3', cls=OVSSwitch)  # Default OVS switch

        # Add hosts
        h1 = self.addHost('h1', ip='10.0.1.1/24')
        h2 = self.addHost('h2', ip='10.0.1.2/24')
        h3 = self.addHost('h3', ip='10.0.1.3/24')

        # Connect hosts to switches
        self.addLink(h1, s1)
        self.addLink(h2, s2)
        self.addLink(h3, s3)

        # Connect switches in a line
        self.addLink(s1, s2)
        self.addLink(s2, s3)

# Start the network with an OVS controller
net = Mininet(topo=SimpleTopo(), controller=OVSController)
net.start()

# Attach the OVS controller to s2 and s3
net.get('s2').start([net.controllers[0]])
net.get('s3').start([net.controllers[0]])

# Run the CLI for debugging
CLI(net)

# Stop the network after exiting CLI
net.stop()


In [ ]:
from mininet.net import Mininet
from mininet.topo import Topo
from mininet.node import RemoteController, OVSSwitch, Host, Switch
from mininet.cli import CLI
from mininet.log import setLogLevel, info
import os

class P4Switch(Switch):
    def __init__(self, name, json_file, thrift_port=9090, **params):
        Switch.__init__(self, name, **params)
        self.json_file = json_file
        self.thrift_port = thrift_port

    def start(self, controllers):
        cmd = f"simple_switch --thrift-port {self.thrift_port} --log-console {self.json_file} &"
        info(f"*** Starting P4 switch {self.name}: {cmd}\n")
        os.system(cmd)

    def stop(self):
        info(f"*** Stopping P4 switch {self.name}\n")
        os.system("pkill simple_switch")

class HybridTopo(Topo):
    def __init__(self, json_file):
        Topo.__init__(self)

        s1 = self.addSwitch('s1', cls=P4Switch, json_file=json_file, thrift_port=9090)
        s2 = self.addSwitch('s2', cls=OVSSwitch)
        s3 = self.addSwitch('s3', cls=OVSSwitch)

        h1 = self.addHost('h1', ip='10.0.1.1/24')
        h2 = self.addHost('h2', ip='10.0.2.1/24')
        h3 = self.addHost('h3', ip='10.0.3.1/24')

        self.addLink(h1, s1)
        self.addLink(h2, s2)
        self.addLink(h3, s3)

        self.addLink(s1, s2)
        self.addLink(s1, s3)

def start_mininet():
    json_file = "basic_forward.json"  # Ensure this file exists
    net = Mininet(topo=HybridTopo(json_file), controller=None)
    net.addController('c0', controller=RemoteController, ip='127.0.0.1', port=6633)

    net.start()
    CLI(net)  # Open CLI within the notebook
    net.stop()

setLogLevel('info')
start_mininet()


### WITH RYU

In [4]:
import os
from mininet.net import Mininet
from mininet.topo import Topo
from mininet.node import RemoteController, OVSSwitch
from mininet.cli import CLI
from mininet.log import setLogLevel, info
import subprocess

class SimpleSwitch(OVSSwitch):
    def __init__(self, name, **params):
        OVSSwitch.__init__(self, name, **params)

class HybridTopo(Topo):
    def __init__(self):
        Topo.__init__(self)

        # Simple switch (s1)
        s1 = self.addSwitch('s1', cls=SimpleSwitch)

        # OVS switches (s2, s3) - OpenFlow switches
        s2 = self.addSwitch('s2', cls=OVSSwitch)
        s3 = self.addSwitch('s3', cls=OVSSwitch)

        # Hosts
        h1 = self.addHost('h1', ip='10.0.1.1/24')
        h2 = self.addHost('h2', ip='10.0.2.1/24')
        h3 = self.addHost('h3', ip='10.0.3.1/24')

        # Connect hosts to their respective switches
        self.addLink(h1, s1)
        self.addLink(h2, s2)
        self.addLink(h3, s3)

        # Connect the switches together
        self.addLink(s1, s2)
        self.addLink(s1, s3)

def start_mininet():

    print("Starting Ryu controller...")
    ryu_process = subprocess.Popen(['ryu-manager', 'ryu.app.simple_switch_13'])
    
    # Start Mininet with the HybridTopo, and use Ryu controller (RemoteController)
    net = Mininet(topo=HybridTopo(), controller=None)

    # Start Ryu controller using subprocess (ensure Ryu is running before Mininet)
    

    # Add OpenFlow controller for s2 and s3
    net.addController('c0', controller=RemoteController, ip='127.0.0.1', port=6633)

    # Start the Mininet network
    net.start()

    # Assign the OpenFlow controller to OVS switches (s2 and s3)
    os.system("sudo ovs-vsctl set-controller s2 tcp:127.0.0.1:6633")
    os.system("sudo ovs-vsctl set-controller s3 tcp:127.0.0.1:6633")

    print("\nNetwork is running! Use the Mininet CLI.")
    CLI(net)

    # Stop Mininet and Ryu controller process after the test
    net.stop()
    ryu_process.terminate()

if __name__ == '__main__':
    setLogLevel('info')
    start_mininet()


*** Creating network
*** Adding hosts:
h1 h2 h3 
*** Adding switches:
s1 s2 s3 
*** Adding links:
(h1, s1) (h2, s2) (h3, s3) (s1, s2) (s1, s3) 
*** Configuring hosts
h1 h2 h3 
Unable to contact the remote controller at 127.0.0.1:6633
*** Starting controller
c0 
*** Starting 3 switches
s1 s2 s3 ......
*** Starting CLI:


Starting Ryu controller...

Network is running! Use the Mininet CLI.


Traceback (most recent call last):
  File "/usr/local/bin/ryu-manager", line 5, in <module>
    from ryu.cmd.manager import main
  File "/usr/local/lib/python3.10/dist-packages/ryu/cmd/manager.py", line 22, in <module>
    from ryu.lib import hub
  File "/usr/local/lib/python3.10/dist-packages/ryu/lib/hub.py", line 30, in <module>
    import eventlet
  File "/usr/local/lib/python3.10/dist-packages/eventlet/__init__.py", line 17, in <module>
    from eventlet import convenience
  File "/usr/local/lib/python3.10/dist-packages/eventlet/convenience.py", line 7, in <module>
    from eventlet.green import socket
  File "/usr/local/lib/python3.10/dist-packages/eventlet/green/socket.py", line 21, in <module>
    from eventlet.support import greendns
  File "/usr/local/lib/python3.10/dist-packages/eventlet/support/greendns.py", line 66, in <module>
    setattr(dns, pkg, import_patched('dns.' + pkg))
  File "/usr/local/lib/python3.10/dist-packages/eventlet/support/greendns.py", line 61, in impor

mininet>  exit


*** Stopping 1 controllers
c0 
*** Stopping 5 links
.....
*** Stopping 3 switches
s1 s2 s3 
*** Stopping 3 hosts
h1 h2 h3 
*** Done


In [3]:
# !pip uninstall eventlet -y
# !pip install eventlet==0.33.1
# !pip install pyyaml netaddr dnspython


In [1]:
import eventlet
import dns
print(eventlet.__version__)
print(dns.__version__)


0.39.0
2.7.0


In [6]:
%%writefile drop_all.p4
#include <core.p4>
#include <v1model.p4>

struct metadata { /* empty */ }

parser MyParser(packet_in packet,
                out headers hdr,
                inout metadata meta,
                inout standard_metadata_t standard_metadata) {
    state start { transition accept; }
}

control MyIngress(inout headers hdr,
                  inout metadata meta,
                  inout standard_metadata_t standard_metadata) {
    apply {
        standard_metadata.egress_spec = 0; // Drop all packets
    }
}

control MyEgress(inout headers hdr,
                 inout metadata meta,
                 inout standard_metadata_t standard_metadata) {
    apply { }
}

control MyDeparser(packet_out packet, in headers hdr) {
    apply { packet.emit(hdr); }
}

V1Switch(MyParser(), MyIngress(), MyEgress(), MyDeparser()) main;

Writing drop_all.p4


In [12]:
from mininet.topo import Topo
from mininet.net import Mininet
from mininet.node import OVSSwitch
from mininet.cli import CLI
from mininet.log import setLogLevel
import subprocess
import time

class P4Switch(OVSSwitch):
    def __init__(self, name, **params):
        super().__init__(name, **params)
        self.sw_path = params.get('sw_path', '/usr/local/bin/simple_switch')
        self.json_path = params.get('json_path', '')
        
    def start(self, controllers):
        # Map BMv2 ports 0 ↔ h1, 1 ↔ s2
        self.cmd(f'{self.sw_path} --log-console \
                -i 0@h1-s1 -i 1@s1-s2 {self.json_path} > /dev/null 2>&1 &')
        time.sleep(10)  # Increase sleep for switch initialization

class NetworkTopo(Topo):
    def build(self):
        # Hosts (Same Subnet)
        h1 = self.addHost('h1', ip='10.0.2.1/24')
        h2 = self.addHost('h2', ip='10.0.2.2/24')
        h3 = self.addHost('h3', ip='10.0.2.3/24')

        # Switches
        s1 = self.addSwitch('s1', cls=P4Switch,
                          sw_path='/usr/local/bin/simple_switch',
                          json_path='basic_forward.json')
        s2 = self.addSwitch('s2', cls=OVSSwitch)
        s3 = self.addSwitch('s3', cls=OVSSwitch)

        # Links with explicit interface names
        self.addLink(h1, s1, intfName1='h1-eth0', intfName2='s1-eth0')
        self.addLink(s1, s2, intfName1='s1-eth1', intfName2='s2-eth0')
        self.addLink(s2, s3)
        self.addLink(s3, h2)
        self.addLink(s3, h3)

if __name__ == '__main__':
    setLogLevel('info')
    
    # Compile P4 program
    subprocess.run(['p4c-bm2-ss', '-o', 'basic_forward.json', 'basic_forward.p4'], check=True)
    
    # Start network
    net = Mininet(topo=NetworkTopo(), controller=None)
    net.start()
    # After net.start()
    net['h1'].cmd('arp -s 10.0.2.2 00:00:00:00:00:02')
    net['h2'].cmd('arp -s 10.0.2.1 00:00:00:00:00:01')
    # Configure OVS switches
    for sw in ['s2', 's3']:
        net[sw].cmd('ovs-ofctl add-flow %s "actions=flood"' % sw)
    
    # Add static ARP entries
    net['h1'].cmd('arp -s 10.0.2.2 00:00:00:00:00:02')
    net['h2'].cmd('arp -s 10.0.2.1 00:00:00:00:00:01')
    
    # Configure P4 switch
    s1 = net['s1']
    # In Jupyter notebook, after starting the network:
    s1.cmd('simple_switch_CLI << EOF\n'
           'mc_mgrp_create 1\n'                # Create multicast group 1
           'mc_node_create 0 0 1 2\n'          # Node 0: ports 0 (h1), 1 (s2), 2 (control)
           'mc_node_associate 1 0\n'            # Associate group 1 with node 0
           'table_add dmac forward 00:00:00:00:00:01 => 0\n'  # h1 → port 0
           'table_add dmac forward 00:00:00:00:00:02 => 1\n'  # h2 → port 1
           'EOF')
    
    # Test connectivity
    print("\n=== Final Connectivity Test ===")
    print("h1 -> h2:", net['h1'].cmd('ping -c 4 10.0.2.2'))
    print("h2 -> h1:", net['h2'].cmd('ping -c 4 10.0.2.1'))
    print("h3 -> h2:", net['h3'].cmd('ping -c 4 10.0.2.2'))
    
    CLI(net)
    net.stop()

*** Creating network
*** Adding hosts:
h1 h2 h3 
*** Adding switches:
s1 s2 s3 
*** Adding links:
(h1, s1) (s1, s2) (s2, s3) (s3, h2) (s3, h3) 
*** Configuring hosts
h1 h2 h3 
*** Starting controller

*** Starting 3 switches
s1 s2 s3 ......



=== Final Connectivity Test ===
h1 -> h2: PING 10.0.2.2 (10.0.2.2) 56(84) bytes of data.

--- 10.0.2.2 ping statistics ---
4 packets transmitted, 0 received, 100% packet loss, time 3105ms


h2 -> h1: PING 10.0.2.1 (10.0.2.1) 56(84) bytes of data.

--- 10.0.2.1 ping statistics ---
4 packets transmitted, 0 received, 100% packet loss, time 3054ms




*** Starting CLI:


h3 -> h2: PING 10.0.2.2 (10.0.2.2) 56(84) bytes of data.
64 bytes from 10.0.2.2: icmp_seq=1 ttl=64 time=0.379 ms
64 bytes from 10.0.2.2: icmp_seq=2 ttl=64 time=0.045 ms
64 bytes from 10.0.2.2: icmp_seq=3 ttl=64 time=0.053 ms
64 bytes from 10.0.2.2: icmp_seq=4 ttl=64 time=0.053 ms

--- 10.0.2.2 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3054ms
rtt min/avg/max/mdev = 0.045/0.132/0.379/0.142 ms



mininet>  h2 ping h3


PING 10.0.2.3 (10.0.2.3) 56(84) bytes of data.
64 bytes from 10.0.2.3: icmp_seq=1 ttl=64 time=0.041 ms
64 bytes from 10.0.2.3: icmp_seq=2 ttl=64 time=0.047 ms
64 bytes from 10.0.2.3: icmp_seq=3 ttl=64 time=0.047 ms

--- 10.0.2.3 ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2067ms
rtt min/avg/max/mdev = 0.041/0.045/0.047/0.003 ms


mininet>  exit


*** Stopping 0 controllers

*** Stopping 5 links
.....
*** Stopping 3 switches
s1 s2 s3 
*** Stopping 3 hosts
h1 h2 h3 
*** Done


##### ALL OVS

In [8]:
from mininet.topo import Topo
from mininet.net import Mininet
from mininet.node import OVSSwitch
from mininet.cli import CLI
from mininet.log import setLogLevel
import subprocess
import time

class NetworkTopo(Topo):
    def build(self):
        # Hosts (Same Subnet)
        h1 = self.addHost('h1', ip='10.0.2.1/24')  # Fixed subnet
        h2 = self.addHost('h2', ip='10.0.2.2/24')
        h3 = self.addHost('h3', ip='10.0.2.3/24')

        # Switches
        s1 = self.addSwitch('s1', cls=OVSSwitch)
        s2 = self.addSwitch('s2', cls=OVSSwitch)
        s3 = self.addSwitch('s3', cls=OVSSwitch)

        # Links
        self.addLink(h1, s1)
        self.addLink(s1, s2)
        self.addLink(s2, s3)
        self.addLink(s3, h2)
        self.addLink(s3, h3)

if __name__ == '__main__':
    setLogLevel('info')
    
    # Start network
    net = Mininet(topo=NetworkTopo(), controller=None)
    net.start()
    
    # Configure OVS switches to flood
    for sw in ['s1', 's2', 's3']:
        net[sw].cmd('ovs-ofctl add-flow %s "actions=flood"' % sw)
    
    # Test connectivity
    print("\n=== Final Connectivity Test ===")
    print("h1 -> h2:", net['h1'].cmd('ping -c 4 10.0.2.2'))  # Now works
    print("h2 -> h1:", net['h2'].cmd('ping -c 4 10.0.2.1'))
    print("h3 -> h2:", net['h3'].cmd('ping -c 4 10.0.2.2'))
    
    CLI(net)
    net.stop()

*** Creating network
*** Adding hosts:
h1 h2 h3 
*** Adding switches:
s1 s2 s3 
*** Adding links:
(h1, s1) (s1, s2) (s2, s3) (s3, h2) (s3, h3) 
*** Configuring hosts
h1 h2 h3 
*** Starting controller

*** Starting 3 switches
s1 s2 s3 ...



=== Final Connectivity Test ===
h1 -> h2: PING 10.0.2.2 (10.0.2.2) 56(84) bytes of data.
64 bytes from 10.0.2.2: icmp_seq=1 ttl=64 time=0.492 ms
64 bytes from 10.0.2.2: icmp_seq=2 ttl=64 time=0.041 ms
64 bytes from 10.0.2.2: icmp_seq=3 ttl=64 time=0.042 ms
64 bytes from 10.0.2.2: icmp_seq=4 ttl=64 time=0.048 ms

--- 10.0.2.2 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3073ms
rtt min/avg/max/mdev = 0.041/0.155/0.492/0.194 ms

h2 -> h1: PING 10.0.2.1 (10.0.2.1) 56(84) bytes of data.
64 bytes from 10.0.2.1: icmp_seq=1 ttl=64 time=0.033 ms
64 bytes from 10.0.2.1: icmp_seq=2 ttl=64 time=0.052 ms
64 bytes from 10.0.2.1: icmp_seq=3 ttl=64 time=0.047 ms
64 bytes from 10.0.2.1: icmp_seq=4 ttl=64 time=0.049 ms

--- 10.0.2.1 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3070ms
rtt min/avg/max/mdev = 0.033/0.045/0.052/0.007 ms



*** Starting CLI:


h3 -> h2: PING 10.0.2.2 (10.0.2.2) 56(84) bytes of data.
64 bytes from 10.0.2.2: icmp_seq=1 ttl=64 time=0.273 ms
64 bytes from 10.0.2.2: icmp_seq=2 ttl=64 time=0.049 ms
64 bytes from 10.0.2.2: icmp_seq=3 ttl=64 time=0.046 ms
64 bytes from 10.0.2.2: icmp_seq=4 ttl=64 time=0.046 ms

--- 10.0.2.2 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3070ms
rtt min/avg/max/mdev = 0.046/0.103/0.273/0.097 ms



mininet>  h1 ping h3


PING 10.0.2.3 (10.0.2.3) 56(84) bytes of data.
64 bytes from 10.0.2.3: icmp_seq=1 ttl=64 time=0.518 ms
64 bytes from 10.0.2.3: icmp_seq=2 ttl=64 time=0.053 ms
64 bytes from 10.0.2.3: icmp_seq=3 ttl=64 time=0.051 ms

--- 10.0.2.3 ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2053ms
rtt min/avg/max/mdev = 0.051/0.207/0.518/0.219 ms


mininet>  exit


*** Stopping 0 controllers

*** Stopping 5 links
.....
*** Stopping 3 switches
s1 s2 s3 
*** Stopping 3 hosts
h1 h2 h3 
*** Done


In [5]:
from mininet.topo import Topo
from mininet.net import Mininet
from mininet.node import OVSSwitch
from mininet.cli import CLI
from mininet.log import setLogLevel
import subprocess
import time

class P4Switch(OVSSwitch):
    def __init__(self, name, **params):
        super().__init__(name, **params)
        self.sw_path = params.get('sw_path', '/usr/local/bin/simple_switch')
        self.json_path = params.get('json_path', '')
        
    def start(self, controllers):
        # BMv2 port mapping (Mininet uses 1-based ports internally)
        self.cmd(f'{self.sw_path} --log-console \
                --thrift-port 9090 \
                -i 1@h1-s1 -i 2@s1-s2 -i 3@s2-s3 -i 4@s3-h2 -i 5@s3-h3 \
                {self.json_path} > /dev/null 2>&1 &')
        time.sleep(10)

class NetworkTopo(Topo):
    def build(self):
        h1 = self.addHost('h1', ip='10.0.2.1/24', mac='00:00:00:00:00:01')
        h2 = self.addHost('h2', ip='10.0.2.2/24', mac='00:00:00:00:00:02')
        h3 = self.addHost('h3', ip='10.0.2.3/24', mac='00:00:00:00:00:03')

        s1 = self.addSwitch('s1', cls=P4Switch,
                            sw_path='/usr/local/bin/simple_switch',
                            json_path='basic_forward.json')
        s2 = self.addSwitch('s2', cls=OVSSwitch)
        s3 = self.addSwitch('s3', cls=OVSSwitch)

        self.addLink(h1, s1)
        self.addLink(s1, s2)
        self.addLink(s2, s3)
        self.addLink(s3, h2)
        self.addLink(s3, h3)

if __name__ == '__main__':
    setLogLevel('info')

    # Compile P4 program
    result = subprocess.run(['p4c-bm2-ss', '-o', 'basic_forward.json', 'basic_forward.p4'], check=True)
    if result.returncode != 0:
        print("P4 compilation failed!")
        exit(1)

    # Start network
    net = Mininet(topo=NetworkTopo(), controller=None)
    net.start()

    # Configure OVS switches
    for sw in ['s2', 's3']:
        net[sw].cmd('ovs-ofctl add-flow %s "actions=flood"' % sw)

    # Configure P4 switch with corrected port mapping
    s1 = net['s1']
    s1.cmd('simple_switch_CLI << EOF\n'
           'mc_mgrp_create 1\n'
           'mc_node_create 0 1 2\n'        
           'mc_node_associate 1 0\n'
           'table_add MyIngress.dmac forward 00:00:00:00:00:02 => 2\n'  # h2 → port 2
           'table_add MyIngress.dmac forward 00:00:00:00:00:01 => 1\n'  # h1 → port 1
           'EOF')
    
    # Set static ARP entries for all hosts
    net['h1'].cmd('arp -s 10.0.2.2 00:00:00:00:00:02')
    net['h2'].cmd('arp -s 10.0.2.1 00:00:00:00:00:01')
    net['h3'].cmd('arp -s 10.0.2.1 00:00:00:00:00:01')
    net['h3'].cmd('arp -s 10.0.2.2 00:00:00:00:00:02')

    # Test connectivity
    print("\n=== Final Connectivity Test ===")
    print("h1 -> h2:", net['h1'].cmd('ping -c 4 10.0.2.2'))
    print("h2 -> h1:", net['h2'].cmd('ping -c 4 10.0.2.1'))

    CLI(net)
    net.stop()


*** Creating network
*** Adding hosts:
h1 h2 h3 
*** Adding switches:
s1 s2 s3 
*** Adding links:
(h1, s1) (s1, s2) (s2, s3) (s3, h2) (s3, h3) 
*** Configuring hosts
h1 h2 h3 
*** Starting controller

*** Starting 3 switches
s1 s2 s3 ......



=== Final Connectivity Test ===
h1 -> h2: PING 10.0.2.2 (10.0.2.2) 56(84) bytes of data.

--- 10.0.2.2 ping statistics ---
4 packets transmitted, 0 received, 100% packet loss, time 3101ms




*** Starting CLI:


h2 -> h1: PING 10.0.2.1 (10.0.2.1) 56(84) bytes of data.

--- 10.0.2.1 ping statistics ---
4 packets transmitted, 0 received, 100% packet loss, time 3054ms




mininet>  exit


*** Stopping 0 controllers

*** Stopping 5 links
.....
*** Stopping 3 switches
s1 s2 s3 
*** Stopping 3 hosts
h1 h2 h3 
*** Done


In [ ]:
from mininet.topo import Topo
from mininet.net import Mininet
from mininet.node import Switch
from mininet.cli import CLI
from mininet.log import setLogLevel
import subprocess
import time
import os

class P4Switch(Switch):
    def __init__(self, name, **params):
        super().__init__(name, **params)
        self.sw_path = params.get('sw_path', '/usr/local/bin/simple_switch')
        self.json_path = params.get('json_path', '')
        self.thrift_port = 9090

    def start(self, controllers):
        # Get actual interface names from Mininet
        intfs = self.intfList()
        if len(intfs) < 2:
            print(f"Error: {self.name} has fewer than 2 interfaces!")
            return
        # Exclude loopback interface
        intfs = [intf for intf in intfs if intf.name != 'lo']
        intf1, intf2 = intfs[0].name, intfs[1].name  # e.g., s1-eth1, s1-eth2

        # Wait for interfaces to be up
        max_wait = 10
        for _ in range(max_wait):
            if (os.path.exists(f"/sys/class/net/{intf1}") and
                os.path.exists(f"/sys/class/net/{intf2}")):
                break
            time.sleep(1)
        else:
            print(f"Interfaces {intf1} and {intf2} not found after {max_wait} seconds!")
            return

        # Start simple_switch
        cmd = f"{self.sw_path} --log-console --thrift-port {self.thrift_port} " \
              f"-i 1@{intf1} -i 2@{intf2} {self.json_path}"
        print(f"Starting P4 switch with command: {cmd}")
        self.cmd(cmd + ' &')
        time.sleep(3)  # Wait for switch to initialize

    def stop(self):
        self.cmd('pkill -f simple_switch')
        super().stop()

class NetworkTopo(Topo):
    def build(self):
        h1 = self.addHost('h1', ip='10.0.2.1/24', mac='00:00:00:00:00:01')
        h2 = self.addHost('h2', ip='10.0.2.2/24', mac='00:00:00:00:00:02')
        h3 = self.addHost('h3', ip='10.0.2.3/24', mac='00:00:00:00:00:03')

        s1 = self.addSwitch('s1', cls=P4Switch,
                            sw_path='/usr/local/bin/simple_switch',
                            json_path='basic_forward.json')
        s2 = self.addSwitch('s2')
        s3 = self.addSwitch('s3')

        self.addLink(h1, s1)
        self.addLink(s1, s2)
        self.addLink(s2, s3)
        self.addLink(s3, h2)
        self.addLink(s3, h3)

if __name__ == '__main__':
    setLogLevel('info')

    # Compile P4 program
    result = subprocess.run(['p4c-bm2-ss', '-o', 'basic_forward.json', 'basic_forward.p4'], check=True)
    if result.returncode != 0:
        print("P4 compilation failed!")
        exit(1)

    # Start network
    net = Mininet(topo=NetworkTopo(), controller=None)
    net.start()

    # Debug: Check interfaces on s1
    s1 = net['s1']
    print("Checking interfaces on s1:")
    intfs = s1.intfList()
    for intf in intfs:
        print(f"Interface {intf}: {s1.cmd(f'ip link show {intf}')}")

    # Configure OVS switches
    for sw in ['s2', 's3']:
        net[sw].cmd('ovs-ofctl add-flow %s "actions=flood"' % sw)

    # Configure P4 switch
    cli_output = s1.cmd(f'simple_switch_CLI --thrift-port 9090 << EOF\n'
                        'mc_mgrp_create 1\n'
                        'mc_node_create 0 1 2\n'
                        'mc_node_associate 1 0\n'
                        'table_add MyIngress.dmac forward 00:00:00:00:00:01 => 1\n'
                        'table_add MyIngress.dmac forward 00:00:00:00:00:02 => 2\n'
                        'table_add MyIngress.dmac forward 00:00:00:00:00:03 => 2\n'
                        'EOF')
    print("CLI Output:", cli_output)

    # Set static ARP entries
    net['h1'].cmd('arp -s 10.0.2.2 00:00:00:00:00:02')
    net['h1'].cmd('arp -s 10.0.2.3 00:00:00:00:00:03')
    net['h2'].cmd('arp -s 10.0.2.1 00:00:00:00:00:01')
    net['h3'].cmd('arp -s 10.0.2.1 00:00:00:00:00:01')
    net['h3'].cmd('arp -s 10.0.2.2 00:00:00:00:00:02')

    # Test connectivity
    print("\n=== Final Connectivity Test ===")
    print("h1 -> h2:", net['h1'].cmd('ping -c 4 10.0.2.2'))
    print("h2 -> h1:", net['h2'].cmd('ping -c 4 10.0.2.1'))
    print("h2 -> h3:", net['h2'].cmd('ping -c 4 10.0.2.3'))

    CLI(net)
    net.stop()

*** Creating network
*** Adding hosts:
h1 h2 h3 
*** Adding switches:
s1 s2 s3 
*** Adding links:
(h1, s1) (s1, s2) (s2, s3) (s3, h2) (s3, h3) 
*** Configuring hosts
h1 h2 h3 
*** Starting controller

*** Starting 3 switches
s1 

Starting P4 switch with command: /usr/local/bin/simple_switch --log-console --thrift-port 9090 -i 1@s1-eth1 -i 2@s1-eth2 basic_forward.json


s2 s3 ...


Checking interfaces on s1:
Interface lo: Calling target program-options parser
[22:30:48.576] [bmv2] [D] [thread 10350] Set default default entry for table 'MyIngress.dmac': MyIngress.flood - 
Adding interface s1-eth1 as port 1
[22:30:48.577] [bmv2] [D] [thread 10350] Adding interface s1-eth1 as port 1
Adding interface s1-eth2 as port 2
[22:30:48.585] [bmv2] [D] [thread 10350] Adding interface s1-eth2 as port 2
[22:30:48.593] [bmv2] [I] [thread 10350] Starting Thrift server on port 9090
[22:30:48.593] [bmv2] [I] [thread 10350] Thrift server was started
[22:30:48.622] [bmv2] [D] [thread 10363] [0.0] [cxt 0] Processing packet received on port 1
[22:30:48.622] [bmv2] [D] [thread 10363] [0.0] [cxt 0] Parser 'parser': start
[22:30:48.622] [bmv2] [D] [thread 10363] [0.0] [cxt 0] Parser 'parser' entering state 'start'
[22:30:48.622] [bmv2] [D] [thread 10363] [0.0] [cxt 0] Extracting header 'ethernet'
[22:30:48.622] [bmv2] [D] [thread 10363] [0.0] [cxt 0] Parser state 'start' has no switch, go

*** Starting CLI:


h2 -> h3: PING 10.0.2.3 (10.0.2.3) 56(84) bytes of data.
64 bytes from 10.0.2.3: icmp_seq=1 ttl=64 time=0.328 ms
64 bytes from 10.0.2.3: icmp_seq=2 ttl=64 time=0.052 ms
64 bytes from 10.0.2.3: icmp_seq=3 ttl=64 time=0.053 ms
64 bytes from 10.0.2.3: icmp_seq=4 ttl=64 time=0.057 ms

--- 10.0.2.3 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3054ms
rtt min/avg/max/mdev = 0.052/0.122/0.328/0.118 ms



In [ ]:
from mininet.topo import Topo
from mininet.net import Mininet
from mininet.node import Switch
from mininet.cli import CLI
from mininet.log import setLogLevel
import subprocess
import time
import os
import json

class P4Switch(Switch):
    def __init__(self, name, **params):
        super().__init__(name, **params)
        self.sw_path = params.get('sw_path', '/usr/local/bin/simple_switch')
        self.json_path = params.get('json_path', '')
        self.thrift_port = 9090
        self.log_file = f"{name}_log.txt"

    def start(self, controllers):
        # Get actual interface names from Mininet
        intfs = self.intfList()
        if len(intfs) < 2:
            print(f"Error: {self.name} has fewer than 2 interfaces!")
            return
        # Exclude loopback interface
        intfs = [intf for intf in intfs if intf.name != 'lo']
        
        # Verify JSON file exists and is valid
        if not os.path.exists(self.json_path):
            print(f"ERROR: JSON file {self.json_path} does not exist!")
            return
            
        try:
            # Validate JSON file
            with open(self.json_path, 'r') as f:
                json.load(f)
            print(f"JSON file {self.json_path} is valid")
        except json.JSONDecodeError as e:
            print(f"ERROR: Invalid JSON file {self.json_path}: {e}")
            print("Attempting to fix JSON file by recompiling...")
            result = subprocess.run(
                ['p4c', '--target', 'bmv2', '--arch', 'v1model', '-o', 'basicforward.json', 'basicforward.p4'], 
                capture_output=True, 
                text=True
            )
            
            if result.returncode != 0:
                print(f"Recompilation failed: {result.stderr}")
                return
            else:
                print("Recompilation successful")
                
        # Get interface names
        intf1, intf2 = intfs[0].name, intfs[1].name  # e.g., s1-eth1, s1-eth2
        print(f"Using interfaces {intf1} and {intf2} for P4 switch")
        
        # Start simple_switch with detailed logging
        cmd = f"{self.sw_path} --log-console --thrift-port {self.thrift_port} " \
              f"-i 1@{intf1} -i 2@{intf2} {self.json_path} > {self.log_file} 2>&1"
              
        print(f"Starting P4 switch with command: {cmd}")
        self.cmd(cmd + ' &')
        
        # Wait for switch to initialize and check status
        time.sleep(3)
        
        # Check if switch is running
        pid = self.cmd(f"pgrep -f 'simple_switch.*{self.thrift_port}'").strip()
        if pid:
            print(f"P4 switch started successfully with PID {pid}")
        else:
            print(f"ERROR: P4 switch failed to start! Check logs in {self.log_file}")
            # Print the logs for debugging
            print("--- Last 20 lines of switch log ---")
            self.cmd(f"tail -20 {self.log_file}")
            print("-----------------------------------")
        # Add control plane configuration
        self.cmd(f'simple_switch_CLI --thrift-port {self.thrift_port} <<EOF\n'
                 'mc_mgrp_create 1\n'
                 'mc_node_create 1 1 2 3\n'
                 'mc_node_associate 1 1\n'
                 'table_add mac_table forward 00:00:00:00:00:01 => 1\n'
                 'table_add mac_table forward 00:00:00:00:00:02 => 2\n'
                 'table_add mac_table forward 00:00:00:00:00:03 => 3\n'
                 'EOF')

    def stop(self):
        self.cmd('pkill -f simple_switch')
        super().stop()

class NetworkTopo(Topo):
    def build(self):
        h1 = self.addHost('h1', ip='10.0.2.1/24', mac='00:00:00:00:00:01')
        h2 = self.addHost('h2', ip='10.0.2.2/24', mac='00:00:00:00:00:02')
        h3 = self.addHost('h3', ip='10.0.2.3/24', mac='00:00:00:00:00:03')

        s1 = self.addSwitch('s1', cls=P4Switch,
                            sw_path='/usr/local/bin/simple_switch',
                            json_path='basic.json')
        s2 = self.addSwitch('s2')
        s3 = self.addSwitch('s3')

        self.addLink(h1, s1)
        self.addLink(s1, s2)
        self.addLink(s2, s3)
        self.addLink(s3, h2)
        self.addLink(s3, h3)

if __name__ == '__main__':
    setLogLevel('info')
    
    # Clean up previous runs
    os.system('sudo pkill -f simple_switch')
    os.system('sudo rm -f *.log *.txt')
    
    # Verify p4c and simple_switch are installed
    if subprocess.run(['which', 'p4c'], capture_output=True).returncode != 0:
        print("ERROR: p4c compiler not found in PATH!")
        exit(1)
        
    if subprocess.run(['which', 'simple_switch'], capture_output=True).returncode != 0:
        print("ERROR: simple_switch not found in PATH!")
        exit(1)
    
    # Compile P4 program with more verbose output
    print("Compiling P4 program...")
    result = subprocess.run(
        ['p4c', '--target', 'bmv2', '--arch', 'v1model', '-o', '.', 'basic.p4'], 
        capture_output=True, 
        text=True
    )
    
    if result.returncode != 0:
        print("P4 compilation failed!")
        print(f"Error: {result.stderr}")
        exit(1)
    else:
        print("P4 compilation successful")
        
    # Validate JSON before starting network
    try:
        with open('basic.json', 'r') as f:
            json.load(f)
        print("JSON output is valid")
    except (json.JSONDecodeError, FileNotFoundError) as e:
        print(f"Error with JSON output: {e}")
        exit(1)

    # Start network
    net = Mininet(topo=NetworkTopo(), controller=None)
    net.start()

    # Debug: Check interfaces on s1
    s1 = net['s1']
    print("Checking interfaces on s1:")
    intfs = s1.intfList()
    for intf in intfs:
        print(f"Interface {intf.name}: {s1.cmd(f'ip link show {intf.name}')}")
    # Configure OVS switches for normal operation
    for sw in ['s2', 's3']:
        net[sw].cmd('ovs-ofctl add-flow %s "actions=flood"' % sw)

    # Test connectivity
    print("\n=== Final Connectivity Test ===")
    print("h1 -> h2:", net['h1'].cmd('ping -c 2 10.0.2.2'))
    print("h2 -> h1:", net['h2'].cmd('ping -c 2 10.0.2.1'))
    print("h2 -> h3:", net['h2'].cmd('ping -c 2 10.0.2.3'))

    # Provide helpful instructions
    print("\n=== Debugging Instructions ===")
    print("1. Check if switch is running: pgrep -f simple_switch")
    print("2. View switch logs: cat s1_log.txt or cat /tmp/switch_debug.log")
    print("3. Try connecting to CLI: simple_switch_CLI --thrift-port 9090")
    print("4. To restart switch directly: sudo simple_switch --log-console --thrift-port 9090 -i 1@s1-eth1 -i 2@s1-eth2 basicforward.json")
    
    CLI(net)
    net.stop()

Compiling P4 program...


*** Creating network
*** Adding hosts:
h1 h2 h3 
*** Adding switches:
s1 s2 s3 
*** Adding links:
(h1, s1) (s1, s2) (s2, s3) (s3, h2) (s3, h3) 
*** Configuring hosts
h1 h2 h3 


P4 compilation successful
JSON output is valid


*** Starting controller

*** Starting 3 switches
s1 

JSON file basic.json is valid
Using interfaces s1-eth1 and s1-eth2 for P4 switch
Starting P4 switch with command: /usr/local/bin/simple_switch --log-console --thrift-port 9090 -i 1@s1-eth1 -i 2@s1-eth2 basic.json > s1_log.txt 2>&1


s2 s3 ...


P4 switch started successfully with PID 15819
Checking interfaces on s1:
Interface lo: 1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN mode DEFAULT group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00

Interface s1-eth1: 176: s1-eth1@if2: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP mode DEFAULT group default qlen 1000
    link/ether 0e:85:b5:25:2b:47 brd ff:ff:ff:ff:ff:ff link-netnsid 0

Interface s1-eth2: 178: s1-eth2@s2-eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP mode DEFAULT group default qlen 1000
    link/ether 9e:40:1f:6a:0c:01 brd ff:ff:ff:ff:ff:ff


=== Final Connectivity Test ===
h1 -> h2: PING 10.0.2.2 (10.0.2.2) 56(84) bytes of data.
From 10.0.2.1 icmp_seq=1 Destination Host Unreachable
From 10.0.2.1 icmp_seq=2 Destination Host Unreachable

--- 10.0.2.2 ping statistics ---
2 packets transmitted, 0 received, +2 errors, 100% packet loss, time 1001ms
pipe 2

h2 -> h1: PING 10.0.2.

*** Starting CLI:


h2 -> h3: PING 10.0.2.3 (10.0.2.3) 56(84) bytes of data.
64 bytes from 10.0.2.3: icmp_seq=1 ttl=64 time=0.395 ms
64 bytes from 10.0.2.3: icmp_seq=2 ttl=64 time=0.055 ms

--- 10.0.2.3 ping statistics ---
2 packets transmitted, 2 received, 0% packet loss, time 1022ms
rtt min/avg/max/mdev = 0.055/0.225/0.395/0.170 ms


=== Debugging Instructions ===
1. Check if switch is running: pgrep -f simple_switch
2. View switch logs: cat s1_log.txt or cat /tmp/switch_debug.log
3. Try connecting to CLI: simple_switch_CLI --thrift-port 9090
4. To restart switch directly: sudo simple_switch --log-console --thrift-port 9090 -i 1@s1-eth1 -i 2@s1-eth2 basicforward.json


In [7]:
!pip list

Package                  Version
------------------------ ----------------
attrs                    21.2.0
Automat                  20.2.0
Babel                    2.8.0
bcrypt                   3.2.0
blinker                  1.4
certifi                  2020.6.20
chardet                  4.0.0
click                    8.0.3
cloud-init               24.4
colorama                 0.4.4
command-not-found        0.3
configobj                5.0.6
constantly               15.1.0
cryptography             3.4.8
dbus-python              1.2.18
debtcollector            3.0.0
distro                   1.7.0
distro-info              1.1+ubuntu0.2
dnspython                2.7.0
ec2-hibinit-agent        1.0.0
eventlet                 0.33.1
googleapis-common-protos 1.67.0
greenlet                 3.1.1
grpcio                   1.70.0
hibagent                 1.0.1
httplib2                 0.20.2
hyperlink                21.0.0
idna                     3.3
importlib-metadata       4.6.4
incremental 

In [8]:
#!pip install thrift

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 KB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for thrift: filename=thrift-0.21.0-cp310-cp310-linux_x86_64.whl size=375186 sha256=f69132227a5a11e247635b9dce332c5795360d888592a403b4c278dd5c811adf
  Stored in directory: /root/.cache/pip/wheels/a6/7f/00/794a649d98a25c862caa95ee7cc28a8b60666cbfaf8dd06a50
Successfully built thrift


# Simplest Topo

The setup here forwards when we add the rules in the table through the network setup.

1. mainfile: forward.p4
2. compiled: forward.json/forward.p4i
3. h1 -> h2 -> h1 is shown.

In [12]:
import os
import subprocess
import time
from mininet.net import Mininet
from mininet.topo import Topo
from mininet.node import Switch
from mininet.cli import CLI
from mininet.log import setLogLevel, info

class P4Switch(Switch):
    def __init__(self, name, json_file, thrift_port=9090, **params):
        Switch.__init__(self, name, **params)
        self.json_file = json_file
        self.thrift_port = thrift_port
        self.interfaces = []  # Store interfaces to bind

    def start(self, controllers):
        # Get interfaces from Mininet (e.g., s1-eth0, s1-eth1)
        intf_args = " ".join(f"--interface {idx}@{intf.name}" for idx, intf in enumerate(self.intfList()) if intf.name != "lo")
        cmd = f"simple_switch --thrift-port {self.thrift_port} --log-console --log-level trace {intf_args} {self.json_file} &"
        info(f"*** Starting P4 switch {self.name}: {cmd}\n")
        os.system(cmd)
        time.sleep(5)  # Wait for switch to bind interfaces

    def stop(self):
        info(f"*** Stopping P4 switch {self.name}\n")
        os.system("pkill simple_switch")

class NetworkTopo(Topo):
    def __init__(self, json_file):
        Topo.__init__(self)
        s1 = self.addSwitch('s1', cls=P4Switch, json_file=json_file, thrift_port=9090)
        h1 = self.addHost('h1', ip='10.0.1.1/24', mac='00:00:00:00:01:01')
        h2 = self.addHost('h2', ip='10.0.1.2/24', mac='00:00:00:00:01:02')
        self.addLink(h1, s1, port1=0, port2=0)
        self.addLink(h2, s1, port1=0, port2=1)

def compile_p4_program():
    print("Compiling P4 program...")
    try:
        subprocess.run(['p4c', '--target', 'bmv2', '--arch', 'v1model', '-o', '.', 'forward.p4'], check=True)
        print("P4 compilation successful")
        time.sleep(5)
    except subprocess.CalledProcessError as e:
        print(f"P4 compilation failed: {e}")
        exit(1)

def setup_network():
    json_file = "forward.json"
    net = Mininet(topo=NetworkTopo(json_file), controller=None)
    try:
        net.start()
        s1 = net.get('s1')
        s1.cmd('simple_switch_CLI --thrift-port 9090 << EOF\n'
               'mc_mgrp_create 1\n'
               'mc_node_create 0 0 1\n'
               'mc_node_associate 1 0\n'
               'EOF')
        s1.cmd('simple_switch_CLI --thrift-port 9090 << EOF\n'
               'table_add ipv4_lpm forward 10.0.1.1/32 => 0\n'
               'table_add ipv4_lpm forward 10.0.1.2/32 => 1\n'
               'EOF')
        h1, h2 = net.get('h1', 'h2')
        h1.cmd('ifconfig h1-eth0 10.0.1.1/24 up')
        h2.cmd('ifconfig h2-eth0 10.0.1.2/24 up')
        h1.cmd('arp -i h1-eth0 -s 10.0.1.2 00:00:00:00:01:02')
        h2.cmd('arp -i h2-eth0 -s 10.0.1.1 00:00:00:00:01:01')
        h1.cmd('ip route add 10.0.1.0/24 dev h1-eth0')
        h2.cmd('ip route add 10.0.1.0/24 dev h2-eth0')
        
        info("*** h1 interface config:\n")
        print(h1.cmd('ifconfig h1-eth0'))
        info("*** h1 ARP table:\n")
        print(h1.cmd('arp -n'))
        info("*** h2 interface config:\n")
        print(h2.cmd('ifconfig h2-eth0'))
        
        info("*** Starting tcpdump on h2-eth0:\n")
        h2.cmd('tcpdump -i h2-eth0 -w h2_capture.pcap &')
        time.sleep(1)
        
        info("*** Pinging from h1 to h2:\n")
        print(h1.cmd('ping -c 10 10.0.1.2'))
        
        h2.cmd('pkill tcpdump')
        info("*** tcpdump output (check h2_capture.pcap manually):\n")
        
        info("*** Pinging h1 to h2 with net.ping:\n")
        print(net.ping([h1, h2]))
        
        CLI(net)
    finally:
        net.stop()

def main():
    setLogLevel('info')
    os.system('sudo pkill -f simple_switch')
    for tool in ['p4c', 'simple_switch', 'simple_switch_CLI']:
        if subprocess.run(['which', tool], capture_output=True).returncode != 0:
            print(f"ERROR: {tool} not found in PATH!")
            exit(1)
    compile_p4_program()
    setup_network()

if __name__ == '__main__':
    main()

Compiling P4 program...
P4 compilation successful


*** Creating network
*** Adding hosts:
h1 h2 
*** Adding switches:
s1 
*** Adding links:
(h1, s1) (h2, s1) 
*** Configuring hosts
h1 h2 
*** Starting controller

*** Starting 1 switches
s1 *** Starting P4 switch s1: simple_switch --thrift-port 9090 --log-console --log-level trace --interface 0@s1-eth0 --interface 1@s1-eth1 forward.json &


Calling target program-options parser
[20:59:55.086] [bmv2] [D] [thread 44106] Set default default entry for table 'MyIngress.ipv4_lpm': MyIngress.drop - 
[20:59:55.086] [bmv2] [D] [thread 44106] Set default default entry for table 'MyIngress.ethernet_exact': MyIngress.broadcast - 
Adding interface s1-eth0 as port 0
[20:59:55.086] [bmv2] [D] [thread 44106] Adding interface s1-eth0 as port 0
Adding interface s1-eth1 as port 1
[20:59:55.095] [bmv2] [D] [thread 44106] Adding interface s1-eth1 as port 1
[20:59:55.105] [bmv2] [I] [thread 44106] Starting Thrift server on port 9090
[20:59:55.105] [bmv2] [I] [thread 44106] Thrift server was started
[20:59:55.318] [bmv2] [D] [thread 44112] [0.0] [cxt 0] Processing packet received on port 0
[20:59:55.318] [bmv2] [D] [thread 44112] [0.0] [cxt 0] Parser 'parser': start
[20:59:55.318] [bmv2] [D] [thread 44112] [0.0] [cxt 0] Parser 'parser' entering state 'start'
[20:59:55.318] [bmv2] [D] [thread 44112] [0.0] [cxt 0] Extracting header 'ethernet'
[20


*** h1 interface config:
*** h1 ARP table:
*** h2 interface config:
*** Starting tcpdump on h2-eth0:


[21:00:00.140] [bmv2] [T] [thread 44120] bm_get_config
[21:00:00.142] [bmv2] [T] [thread 44120] bm_mc_mgrp_create
[21:00:00.142] [bmv2] [D] [thread 44120] mgrp node created for mgid 1
[21:00:00.142] [bmv2] [T] [thread 44120] bm_mc_node_create
[21:00:00.142] [bmv2] [D] [thread 44120] node created for rid 0
[21:00:00.142] [bmv2] [T] [thread 44120] bm_mc_node_associate
[21:00:00.142] [bmv2] [D] [thread 44120] node associated with mgid 1
[21:00:00.205] [bmv2] [T] [thread 44122] bm_get_config
[21:00:00.206] [bmv2] [T] [thread 44122] bm_table_add_entry
[21:00:00.206] [bmv2] [D] [thread 44122] Entry 0 added to table 'MyIngress.ipv4_lpm'
[21:00:00.206] [bmv2] [D] [thread 44122] Dumping entry 0
Match key:
* hdr.ipv4.dstAddr    : LPM       0a000101/32
Action entry: MyIngress.forward - 0,

[21:00:00.207] [bmv2] [T] [thread 44122] bm_table_add_entry
[21:00:00.207] [bmv2] [D] [thread 44122] Entry 1 added to table 'MyIngress.ipv4_lpm'
[21:00:00.207] [bmv2] [D] [thread 44122] Dumping entry 1
Match ke

...

[21:00:01.150] [bmv2] [D] [thread 44112] [19.0] [cxt 0] Processing packet received on port 0
[21:00:01.150] [bmv2] [D] [thread 44112] [19.0] [cxt 0] Parser 'parser': start
[21:00:01.150] [bmv2] [D] [thread 44112] [19.0] [cxt 0] Parser 'parser' entering state 'start'
[21:00:01.150] [bmv2] [D] [thread 44112] [19.0] [cxt 0] Extracting header 'ethernet'
[21:00:01.150] [bmv2] [D] [thread 44112] [19.0] [cxt 0] Parser state 'start': key is 86dd
[21:00:01.150] [bmv2] [T] [thread 44112] [19.0] [cxt 0] Bytes parsed: 14
[21:00:01.150] [bmv2] [D] [thread 44112] [19.0] [cxt 0] Parser 'parser': end
[21:00:01.150] [bmv2] [D] [thread 44112] [19.0] [cxt 0] Pipeline 'ingress': start
[21:00:01.150] [bmv2] [T] [thread 44112] [19.0] [cxt 0] forward.p4(106) Condition "hdr.ipv4.isValid()" (node_2) is false
[21:00:01.150] [bmv2] [T] [thread 44112] [19.0] [cxt 0] forward.p4(108) Condition "hdr.ethernet.isValid()" (node_4) is true
[21:00:01.150] [bmv2] [T] [thread 44112] [19.0] [cxt 0] Applying table 'MyIngress

*** tcpdump output (check h2_capture.pcap manually):
*** Pinging h1 to h2 with net.ping:
h1 -> h2 
h2 -> h1 
*** Results: 0% dropped (2/2 received)
*** Starting CLI:


[21:00:10.430] [bmv2] [D] [thread 44112] [44.0] [cxt 0] Processing packet received on port 0
[21:00:10.430] [bmv2] [D] [thread 44112] [44.0] [cxt 0] Parser 'parser': start
[21:00:10.430] [bmv2] [D] [thread 44112] [44.0] [cxt 0] Parser 'parser' entering state 'start'
[21:00:10.430] [bmv2] [D] [thread 44112] [44.0] [cxt 0] Extracting header 'ethernet'
[21:00:10.430] [bmv2] [D] [thread 44112] [44.0] [cxt 0] Parser state 'start': key is 0800
[21:00:10.430] [bmv2] [T] [thread 44112] [44.0] [cxt 0] Bytes parsed: 14
[21:00:10.430] [bmv2] [D] [thread 44112] [44.0] [cxt 0] Parser 'parser' entering state 'parse_ipv4'
[21:00:10.430] [bmv2] [D] [thread 44112] [44.0] [cxt 0] Extracting header 'ipv4'
[21:00:10.430] [bmv2] [D] [thread 44112] [44.0] [cxt 0] Parser state 'parse_ipv4' has no switch, going to default next state
[21:00:10.430] [bmv2] [T] [thread 44112] [44.0] [cxt 0] Bytes parsed: 34
[21:00:10.430] [bmv2] [D] [thread 44112] [44.0] [cxt 0] Parser 'parser': end
[21:00:10.430] [bmv2] [D] [th

mininet>  exit


*** Stopping 0 controllers

*** Stopping 2 links
..
*** Stopping 1 switches
s1 *** Stopping P4 switch s1

*** Stopping 2 hosts
h1 h2 
*** Done


# Dropping

In [11]:
import os
import subprocess
import time
from mininet.net import Mininet
from mininet.topo import Topo
from mininet.node import Switch
from mininet.cli import CLI
from mininet.log import setLogLevel, info

class P4Switch(Switch):
    def __init__(self, name, json_file, thrift_port=9090, **params):
        Switch.__init__(self, name, **params)
        self.json_file = json_file
        self.thrift_port = thrift_port
        self.interfaces = []  # Store interfaces to bind

    def start(self, controllers):
        # Get interfaces from Mininet (e.g., s1-eth0, s1-eth1)
        intf_args = " ".join(f"--interface {idx}@{intf.name}" for idx, intf in enumerate(self.intfList()) if intf.name != "lo")
        cmd = f"simple_switch --thrift-port {self.thrift_port} --log-console --log-level trace {intf_args} {self.json_file} &"
        info(f"*** Starting P4 switch {self.name}: {cmd}\n")
        os.system(cmd)
        time.sleep(5)  # Wait for switch to bind interfaces

    def stop(self):
        info(f"*** Stopping P4 switch {self.name}\n")
        os.system("pkill simple_switch")

class NetworkTopo(Topo):
    def __init__(self, json_file):
        Topo.__init__(self)
        s1 = self.addSwitch('s1', cls=P4Switch, json_file=json_file, thrift_port=9090)
        h1 = self.addHost('h1', ip='10.0.1.1/24', mac='00:00:00:00:01:01')
        h2 = self.addHost('h2', ip='10.0.1.2/24', mac='00:00:00:00:01:02')
        self.addLink(h1, s1, port1=0, port2=0)
        self.addLink(h2, s1, port1=0, port2=1)

def compile_p4_program():
    print("Compiling P4 program...")
    try:
        subprocess.run(['p4c', '--target', 'bmv2', '--arch', 'v1model', '-o', '.', 'drop.p4'], check=True)
        print("P4 compilation successful")
        time.sleep(5)
    except subprocess.CalledProcessError as e:
        print(f"P4 compilation failed: {e}")
        exit(1)

def setup_network():
    json_file = "drop.json"
    net = Mininet(topo=NetworkTopo(json_file), controller=None)
    try:
        net.start()
        s1 = net.get('s1')
        s1.cmd('simple_switch_CLI --thrift-port 9090 << EOF\n'
               'mc_mgrp_create 1\n'
               'mc_node_create 0 0 1\n'
               'mc_node_associate 1 0\n'
               'EOF')
        s1.cmd('simple_switch_CLI --thrift-port 9090 << EOF\n'
               'table_add ipv4_filter drop 10.0.1.1 10.0.1.2 =>\n'    # Drop h1 to h2
               'table_add ipv4_filter forward 10.0.1.2 10.0.1.1 => 0\n'  # h2 to h1
               'table_add ipv4_filter forward 10.0.1.1 10.0.1.1 => 0\n'  # h1 to itself
               'table_add ipv4_filter forward 10.0.1.2 10.0.1.2 => 1\n'  # h2 to itself
               'EOF')
        h1, h2 = net.get('h1', 'h2')
        h1.cmd('ifconfig h1-eth0 10.0.1.1/24 up')
        h2.cmd('ifconfig h2-eth0 10.0.1.2/24 up')
        h1.cmd('arp -i h1-eth0 -s 10.0.1.2 00:00:00:00:01:02')
        h2.cmd('arp -i h2-eth0 -s 10.0.1.1 00:00:00:00:01:01')
        h1.cmd('ip route add 10.0.1.0/24 dev h1-eth0')
        h2.cmd('ip route add 10.0.1.0/24 dev h2-eth0')
        
        info("*** h1 interface config:\n")
        print(h1.cmd('ifconfig h1-eth0'))
        info("*** h1 ARP table:\n")
        print(h1.cmd('arp -n'))
        info("*** h2 interface config:\n")
        print(h2.cmd('ifconfig h2-eth0'))
        
        info("*** Starting tcpdump on h2-eth0:\n")
        h2.cmd('tcpdump -i h2-eth0 -w h2_capture.pcap &')
        time.sleep(1)
        
        info("*** Pinging from h1 to h2 (should fail):\n")
        print(h1.cmd('ping -c 10 10.0.1.2'))
        
        info("*** Pinging from h2 to h1 (should succeed):\n")
        print(h2.cmd('ping -c 10 10.0.1.1'))
        
        h2.cmd('pkill tcpdump')
        info("*** tcpdump output (check h2_capture.pcap manually):\n")
        
        info("*** Pinging h1 to h2 with net.ping (mixed results expected):\n")
        print(net.ping([h1, h2]))
        
        CLI(net)
    finally:
        net.stop()

def main():
    setLogLevel('info')
    os.system('sudo pkill -f simple_switch')
    for tool in ['p4c', 'simple_switch', 'simple_switch_CLI']:
        if subprocess.run(['which', tool], capture_output=True).returncode != 0:
            print(f"ERROR: {tool} not found in PATH!")
            exit(1)
    compile_p4_program()
    setup_network()

if __name__ == '__main__':
    main()

Compiling P4 program...
P4 compilation successful


*** Creating network
*** Adding hosts:
h1 h2 
*** Adding switches:
s1 
*** Adding links:
(h1, s1) (h2, s1) 
*** Configuring hosts
h1 h2 
*** Starting controller

*** Starting 1 switches
s1 *** Starting P4 switch s1: simple_switch --thrift-port 9090 --log-console --log-level trace --interface 0@s1-eth0 --interface 1@s1-eth1 drop.json &


Calling target program-options parser
[20:56:43.312] [bmv2] [D] [thread 43869] Set default default entry for table 'MyIngress.ipv4_filter': MyIngress.forward - 0,
[20:56:43.312] [bmv2] [D] [thread 43869] Set default default entry for table 'MyIngress.ethernet_exact': MyIngress.broadcast - 
Adding interface s1-eth0 as port 0
[20:56:43.313] [bmv2] [D] [thread 43869] Adding interface s1-eth0 as port 0
Adding interface s1-eth1 as port 1
[20:56:43.322] [bmv2] [D] [thread 43869] Adding interface s1-eth1 as port 1
[20:56:43.332] [bmv2] [I] [thread 43869] Starting Thrift server on port 9090
[20:56:43.332] [bmv2] [I] [thread 43869] Thrift server was started
[20:56:43.341] [bmv2] [D] [thread 43875] [0.0] [cxt 0] Processing packet received on port 0
[20:56:43.341] [bmv2] [D] [thread 43875] [0.0] [cxt 0] Parser 'parser': start
[20:56:43.341] [bmv2] [D] [thread 43875] [0.0] [cxt 0] Parser 'parser' entering state 'start'
[20:56:43.341] [bmv2] [D] [thread 43875] [0.0] [cxt 0] Extracting header 'ether


*** h1 interface config:
*** h1 ARP table:
*** h2 interface config:
*** Starting tcpdump on h2-eth0:


[20:56:48.319] [bmv2] [D] [thread 43875] [19.0] [cxt 0] Processing packet received on port 0
[20:56:48.319] [bmv2] [D] [thread 43875] [19.0] [cxt 0] Parser 'parser': start
[20:56:48.319] [bmv2] [D] [thread 43875] [19.0] [cxt 0] Parser 'parser' entering state 'start'
[20:56:48.319] [bmv2] [D] [thread 43875] [19.0] [cxt 0] Extracting header 'ethernet'
[20:56:48.319] [bmv2] [D] [thread 43875] [19.0] [cxt 0] Parser state 'start': key is 86dd
[20:56:48.319] [bmv2] [T] [thread 43875] [19.0] [cxt 0] Bytes parsed: 14
[20:56:48.319] [bmv2] [D] [thread 43875] [19.0] [cxt 0] Parser 'parser': end
[20:56:48.319] [bmv2] [D] [thread 43875] [19.0] [cxt 0] Pipeline 'ingress': start
[20:56:48.319] [bmv2] [T] [thread 43875] [19.0] [cxt 0] drop.p4(87) Condition "hdr.ipv4.isValid()" (node_2) is false
[20:56:48.319] [bmv2] [T] [thread 43875] [19.0] [cxt 0] drop.p4(89) Condition "hdr.ethernet.isValid()" (node_4) is true
[20:56:48.319] [bmv2] [T] [thread 43875] [19.0] [cxt 0] Applying table 'MyIngress.etherne

...

100.0


mininet>  links


h1-eth0<->s1-eth0 (OK OK) 
h2-eth0<->s1-eth1 (OK OK) 


mininet>  exit


*** Stopping 0 controllers

*** Stopping 2 links
..
*** Stopping 1 switches
s1 *** Stopping P4 switch s1

*** Stopping 2 hosts
h1 h2 
*** Done


In [14]:
from scapy.all import rdpcap
import pandas as pd

# Function to extract relevant packet details
def extract_packet_info(pcap_file):
    packets = rdpcap(pcap_file)  # Read the PCAP file
    packet_data = []

    for pkt in packets:
        packet_info = {}
        
        # Basic packet info
        packet_info['timestamp'] = float(pkt.time)  # Ensure timestamp is a float
        packet_info['length'] = len(pkt)            # Packet length in bytes
        
        # Ethernet layer (if present)
        if 'Ethernet' in pkt:
            packet_info['src_mac'] = pkt['Ethernet'].src
            packet_info['dst_mac'] = pkt['Ethernet'].dst
            packet_info['eth_type'] = hex(pkt['Ethernet'].type)

        # IP layer (if present)
        if 'IP' in pkt:
            packet_info['src_ip'] = pkt['IP'].src
            packet_info['dst_ip'] = pkt['IP'].dst
            packet_info['protocol'] = pkt['IP'].proto  # e.g., 1=ICMP, 6=TCP, 17=UDP
            packet_info['ttl'] = pkt['IP'].ttl

        # ICMP layer (if present, for ping)
        if 'ICMP' in pkt:
            packet_info['icmp_type'] = pkt['ICMP'].type  # e.g., 8=request, 0=reply
            packet_info['icmp_code'] = pkt['ICMP'].code

        # Add packet info to list
        packet_data.append(packet_info)

    return packet_data

# Main execution
if __name__ == "__main__":
    # Specify your PCAP file
    pcap_file = "h2_capture.pcap"  # Adjust path if needed
    
    try:
        # Extract packet data
        packet_data = extract_packet_info(pcap_file)
        
        # Convert to Pandas DataFrame
        df = pd.DataFrame(packet_data)
        
        # Convert timestamp to human-readable format
        if not df.empty and 'timestamp' in df.columns:
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', errors='coerce')
        
        # Display the DataFrame
        print("Packet DataFrame:")
        print(df)
        
        # Basic analysis
        print("\nSummary Statistics:")
        print(df.describe())
        
        # Filter for ICMP packets (ping traffic)
        if 'protocol' in df.columns:
            icmp_packets = df[df['protocol'] == 1]  # ICMP protocol number is 1
            print("\nICMP Packets:")
            print(icmp_packets[['timestamp', 'src_ip', 'dst_ip', 'icmp_type', 'icmp_code']])
        
        # Optional: Save to CSV for further analysis
        df.to_csv("h2_capture.csv", index=False)
        print("\nSaved to h2_capture.csv")
        
    except Exception as e:
        print(f"Error processing PCAP file: {e}")

Packet DataFrame:
                       timestamp  length            src_mac  \
0  2025-03-04 21:00:01.150630951      70  00:00:00:00:01:02   
1  2025-03-04 21:00:01.150639057      70  a2:12:bc:a5:b2:28   
2  2025-03-04 21:00:01.151190996      70  52:cc:09:43:3c:02   
3  2025-03-04 21:00:01.151309967      70  a2:12:bc:a5:b2:28   
4  2025-03-04 21:00:01.151433945      70  00:00:00:00:01:02   
5  2025-03-04 21:00:01.235028028      98  00:00:00:00:01:01   
6  2025-03-04 21:00:01.235049963      98  00:00:00:00:01:02   
7  2025-03-04 21:00:01.662961960      70  00:00:00:00:01:01   
8  2025-03-04 21:00:02.238919020      98  00:00:00:00:01:01   
9  2025-03-04 21:00:02.238939047      98  00:00:00:00:01:02   
10 2025-03-04 21:00:03.262892962      98  00:00:00:00:01:01   
11 2025-03-04 21:00:03.262913942      98  00:00:00:00:01:02   
12 2025-03-04 21:00:04.287003994      98  00:00:00:00:01:01   
13 2025-03-04 21:00:04.287020922      98  00:00:00:00:01:02   
14 2025-03-04 21:00:05.310969114     